# First Frontier LLM Project

## Import the necessary libraries

In [4]:
import os
from openai import OpenAI
import requests
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display

In [12]:
current_path = os.getcwd()
grandparent = os.path.dirname(os.path.dirname(current_path))
grandparent

'/mnt/e/Development/Python/LLMEngineering/LLMEngineeringPractice'

In [13]:
# Load environment variabls
env_path = grandparent + "/.env"
load_dotenv(env_path,override=True)
api_key = os.getenv('OPENAI_API_KEY')

# check the key
if not api_key:
    print("No API key was found - please head over to the troubleshooting notebook in this folder to identify & fix!")
elif not api_key.startswith("sk-proj-"):
    print("An API key was found, but it doesn't start sk-proj-; please check you're using the right key - see troubleshooting notebook")
elif api_key.strip() != api_key:
    print("An API key was found, but it looks like it might have space or tab characters at the start or end - please remove them - see troubleshooting notebook")
else:
    print("API key found and looks good so far!")

API key found and looks good so far!


## Call OpenAI API to generate text

In [15]:
openai = OpenAI()

message = "Hello, my name is Duy Khang. This is my first ever message to you! Nice to meet you!"
response = openai.chat.completions.create(model="gpt-4o-mini", messages=[{"role":"user", "content":message}])
print(response.choices[0].message.content)

Hello, Duy Khang! It's great to meet you too! How can I assist you today?
